In [1]:
import numpy as np

In [2]:
matches_query = np.load('data/matches_query.npy', allow_pickle=True).item()
len(matches_query)

5

In [3]:
scene_fingerprints = np.load('data/scene_fingerprints.npy')

In [104]:
scene_fingerprints[scene_fingerprints[:, 1] == 12394]
#492 - 486
#len(matches_query['aggregations']['scenes']['buckets'])

array([[  933, 12394]])

In [102]:
matches_query['aggregations']['scenes']['buckets'][2]['top_hits']['hits']['hits']

[{'_index': 'fingerprints_patterns',
  '_type': '_doc',
  '_id': 'OU2-wnQBLzZZGVzjBi1-',
  '_score': 0.0,
  '_source': {'offset': 344, 'hash': 16427}},
 {'_index': 'fingerprints_patterns',
  '_type': '_doc',
  '_id': 'RU2-wnQBLzZZGVzjBi1-',
  '_score': 0.0,
  '_source': {'offset': 366, 'hash': 16427}},
 {'_index': 'fingerprints_patterns',
  '_type': '_doc',
  '_id': '4k2-wnQBLzZZGVzjBit-',
  '_score': 0.0,
  '_source': {'offset': 23, 'hash': 12394}},
 {'_index': 'fingerprints_patterns',
  '_type': '_doc',
  '_id': 'VE2-wnQBLzZZGVzjBix-',
  '_score': 0.0,
  '_source': {'offset': 86, 'hash': 16427}},
 {'_index': 'fingerprints_patterns',
  '_type': '_doc',
  '_id': '702-wnQBLzZZGVzjBix-',
  '_score': 0.0,
  '_source': {'offset': 280, 'hash': 16427}},
 {'_index': 'fingerprints_patterns',
  '_type': '_doc',
  '_id': 'AU2-wnQBLzZZGVzjBi1-',
  '_score': 0.0,
  '_source': {'offset': 302, 'hash': 16427}},
 {'_index': 'fingerprints_patterns',
  '_type': '_doc',
  '_id': 'hk2-wnQBLzZZGVzjBix-',
 

In [107]:
def score_with_offset(matches_query, fingerprints, dont_filter=False):
    #total_candidates = len(matches_query['aggregations']['scenes']['buckets'])
    result = []
    #is_first = True
    for candidates in matches_query['aggregations']['scenes']['buckets']:
        freq = {}
        #x = 0
        for hit in candidates['top_hits']['hits']['hits']:
            #x += 1
            match_hash = hit['_source']['hash']
            match_offset = hit['_source']['offset']
            #print(match_offset, scene_fingerprints[scene_fingerprints[:, 1] == match_hash][0][0], scene_fingerprints[scene_fingerprints[:, 1] == match_hash][0][0] - match_offset)
            delta_offset = scene_fingerprints[scene_fingerprints[:, 1] == match_hash][0][0] - match_offset
            if delta_offset in freq:
                freq[delta_offset] += 1
            else:
                freq[delta_offset] = 1
        #print(x)
        #print(freq)
        offset = max(freq, key=freq.get)
        offset_count = freq[offset]
        key = candidates['key']
        offset_score = offset_count/candidates['bg_count']
        result.append({'key' : key, 'offset' : offset, 'offset_count' : offset_count,
                       'offset_score' : offset_score })
        #is_first = False
    
            
    return result
        
score_with_offset(matches_query, scene_fingerprints)
            

[{'key': 'DjvCu3QBoaf2Y_2A3bmT',
  'offset': 6,
  'offset_count': 212,
  'offset_score': 0.1774058577405858},
 {'key': 'QpwOvHQBdT-cPgIEckxS',
  'offset': 15,
  'offset_count': 198,
  'offset_score': 0.1752212389380531},
 {'key': '1MtUpXQBXtu81SlS8nsP',
  'offset': 244,
  'offset_count': 1,
  'offset_score': 0.006329113924050633},
 {'key': 'lYJFpHQBOaObx7xaWbv5',
  'offset': 391,
  'offset_count': 1,
  'offset_score': 0.005681818181818182},
 {'key': 'WkEslXQBkSohu0M7zw3h',
  'offset': -18,
  'offset_count': 2,
  'offset_score': 0.0035026269702276708},
 {'key': 'lWGJqXQBE5vHt1SDbQAU',
  'offset': 386,
  'offset_count': 2,
  'offset_score': 0.0018298261665141812},
 {'key': 'epIdqHQB3dj8gDV29z8G',
  'offset': 287,
  'offset_count': 1,
  'offset_score': 0.005813953488372093},
 {'key': 'oR28onQBMd_8Sl4jtDNj',
  'offset': 453,
  'offset_count': 2,
  'offset_score': 0.003780718336483932},
 {'key': '5ErZpXQB1fGddnL2pC6x',
  'offset': -302,
  'offset_count': 2,
  'offset_score': 0.0038610038610